In [1]:
import csv
import glob
import os
import os.path
from subprocess import call

In [2]:
data_file = []
folders = ['classified_data/train', 'classified_data/test']

for folder in folders:
    class_folders = glob.glob(os.path.join(folder, '*'))
    for video_class in class_folders:
        class_files = glob.glob(os.path.join(video_class , '*.avi'))
        for video_file in class_files:
            parts = video_file.split(os.path.sep)
            filename = parts[3]
            filename_no_ext = filename.split('.')[0]
            classname = parts[2]
            train_or_test = parts[1]
            if not os.path.exists(os.path.join('classified_data', train_or_test, classname, filename_no_ext + '-0001.jpg')):
                src = os.path.join('classified_data' , train_or_test, classname, filename)
                if not os.path.exists(os.path.join('classified_data', train_or_test, classname, filename_no_ext)):
                    os.makedirs(os.path.join('classified_data', train_or_test, classname, filename_no_ext))
                dest = os.path.join('classified_data' ,train_or_test, classname, filename_no_ext, filename_no_ext + '-%04d.jpg')
                call(['ffmpeg', "-i", src, dest])
                generated_files = glob.glob(os.path.join('classified_data', train_or_test, classname, filename_no_ext, filename_no_ext + '*.jpg'))
                number_frames = len(generated_files)
                data_file.append([train_or_test, classname, filename_no_ext, number_frames])
print(data_file)

[['train', 'Golf Swing', '3283-8_700741', 60], ['train', 'Golf Swing', '3283-8_701201', 60], ['train', 'Golf Swing', '7606-2_700810', 60], ['train', 'Golf Swing', '7608-12_70275', 60], ['train', 'Golf Swing', '7608-5_70039', 17], ['train', 'Golf Swing', '7608-5_70308', 60], ['train', 'Golf Swing', '7608-9_70(2)151', 60], ['train', 'Golf Swing', 'RF1-13157_70040', 60], ['train', 'Golf Swing', 'RF1-13428_70288', 60], ['train', 'Golf Swing', 'RF1-13588_70046', 60], ['train', 'Golf Swing', 'RF1-13678_70045', 60], ['train', 'Golf Swing', 'RF1-13903_70070', 60], ['train', 'Kicking', '1084-44_l865', 23], ['train', 'Kicking', '3833-27_70006', 23], ['train', 'Kicking', '3833-40_70336', 23], ['train', 'Kicking', '3833-41_70187', 23], ['train', 'Kicking', '3937-9_70105', 23], ['train', 'Kicking', '3949-12_70207', 23], ['train', 'Kicking', '5117-8_70113', 23], ['train', 'Kicking', '5117-8_70350', 23], ['train', 'Kicking', '5863-29_70025', 23], ['train', 'Kicking', '6063-20_70333', 23], ['train', '

In [3]:
with open('classified_data/data.csv','w') as fout:
    writer = csv.writer(fout)
    writer.writerows(data_file)

In [4]:
with open('classified_data/data.csv', 'r') as fin:
    reader = csv.reader(fin)
    data_file = list(reader)

In [5]:
import numpy as np
from tqdm import tqdm

In [6]:
frames_considered = 20
classes = []
for item in data_file:
    if item[1] not in classes:
        classes.append(item[1])
classes = sorted(classes)
print(classes)

['Golf Swing', 'Kicking', 'Lifting', 'Riding Horse', 'Running', 'SkateBoarding', 'Swing-Bench', 'Swing-Side', 'Walking']


In [7]:
clean_data = data_file

In [8]:
from keras.preprocessing import image
from keras.applications.inception_v3 import InceptionV3, preprocess_input
from keras.models import Model

/home/neelr/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [9]:
seq_model = InceptionV3(weights='imagenet', include_top=True)

In [11]:
from keras.preprocessing import image
pbar = tqdm(total = len(clean_data))
if not os.path.exists(os.path.join('classified_data', 'sequences')):
    os.makedirs(os.path.join('classified_data', 'sequences'))
for item in clean_data:
    output_frames = []
    path = os.path.join('classified_data', 'sequences', item[2] + '-' + str(frames_considered) + '-features')
    if os.path.isfile(path + '.npy'):
        pbar.update(1)
        continue
    else:
        os.mknod(path+'.npy')
    frames_path = os.path.join('classified_data', item[0], item[1])
    file_name = item[2]
    frames = sorted(glob.glob(os.path.join(frames_path, file_name, file_name + '*.jpg')))
    
    if (int)(item[3]) < frames_considered:
        reps = (int)(frames_considered/len(frames))
        remainder = (frames_considered%len(frames))
        startreps = reps + (int)(remainder/2)
        endreps = reps + (int)(remainder - startreps + reps)
        frame_number  = 0
        #print(str(reps) + " " +str(remainder) + " " + str(startreps) + " " + str(endreps))
        #c = 0
        for frame in frames:
            if frame_number == 0:
                for i in range(0, startreps):
                    output_frames.append(frame)
                    #c += 1
                    #print(c)
            elif frame_number == (len(frames) - 1):
                for i in range(0, endreps):
                    output_frames.append(frame)
                    #c += 1
                    #print(c)
            else:
                for i in range(0,reps):
                    output_frames.append(frame)
                    #c += 1
                    #print(c)
            frame_number += 1
    else:
        skip = len(frames) // frames_considered
        output_frames = [frames[i] for i in range(0, len(frames), skip)]
    sequence = []
    count = 0
    #print(item[2] + " " + (str)(len(output_frames)))
    for frame in output_frames:
        if count < frames_considered:
            img = image.load_img(frame, target_size=(299, 299))
            x = image.img_to_array(img)
            x = np.expand_dims(x, axis = 0)
            x = preprocess_input(x)
            features = seq_model.predict(x)
            features = features[0]
            count += 1
            sequence.append(features)
    #print('number of frames', count)
    np.save(path, sequence)
    pbar.update(1)
pbar.close()

  4%|▎         | 4/109 [00:35<16:01,  9.16s/it]

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20


 34%|███▍      | 37/109 [06:10<11:21,  9.47s/it]

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20


 60%|█████▉    | 65/109 [10:35<07:09,  9.76s/it]

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20


 62%|██████▏   | 68/109 [11:05<06:48,  9.98s/it]

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20


100%|██████████| 109/109 [16:52<00:00,  9.27s/it]


In [12]:
from keras.layers import LSTM, Dense
from keras.models import Sequential

In [12]:
model = Sequential()
model.add(LSTM(128, input_shape=(frames_considered, 1000), return_sequences = True))
model.add(LSTM(128))
model.add(Dense(len(classes), activation='softmax'))

In [13]:
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics=['accuracy'])

In [14]:
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 15, 128)           578048    
_________________________________________________________________
lstm_2 (LSTM)                (None, 128)               131584    
_________________________________________________________________
dense_1 (Dense)              (None, 9)                 1161      
Total params: 710,793
Trainable params: 710,793
Non-trainable params: 0
_________________________________________________________________
None


In [13]:
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, TensorBoard

In [14]:
callbacks = [EarlyStopping(monitor='accuracy', patience = 8, verbose=1, min_delta=1e-4, mode='max'),
             ReduceLROnPlateau(monitor='accuracy', factor=0.1, patience=4, verbose=1, epsilon=1e-4,mode='max')]

In [15]:
train_dataset = []
test_dataset = []
for item in clean_data:
    if item[0] == 'train':
        train_dataset.append(item)
    else:
        test_dataset.append(item)

import random
from keras.utils import to_categorical
def frame_generator(batch_size,train_test):
    current_data = train_dataset if train_test == 'train' else test_dataset
    while 1:
        x, y = [] , []
        for _ in range(batch_size):
            sequence = None
            sample = random.choice(current_data)
            filename = sample[2]
            path = os.path.join('classified_data','sequences',filename+'-' + (str)(frames_considered) + '-features.npy' )
            if os.path.isfile(path):
                sequence = np.load(path)
            label_encoded = classes.index(sample[1])
            label_hot = to_categorical(label_encoded, len(classes))
            np.reshape(sequence, (sequence.shape[0], 1, sequence.shape[1]))
            #print(sequence.shape)
            x.append(sequence)
            y.append(label_hot)
            #x.reshape(40, 2048)
            yield np.array(x), np.array(y)

In [16]:
train_generator = frame_generator(32, 'train')
test_generator = frame_generator(32, 'test')

In [19]:
epochs = 10
model.fit_generator(generator=train_generator, epochs = 10, steps_per_epoch=50, callbacks=callbacks, verbose=1,
                   validation_data=test_generator, validation_steps=50)

Epoch 1/10
50/50 [==============================] - 5s 102ms/step - loss: 1.5834 - acc: 0.5391 - val_loss: 1.8524 - val_acc: 0.4592
Epoch 2/10
 1/50 [..............................] - ETA: 2s - loss: 1.0996 - acc: 0.5789

/home/neelr/anaconda3/lib/python3.6/site-packages/keras/callbacks.py:526: RuntimeWarning: Early stopping conditioned on metric `accuracy` which is not available. Available metrics are: val_loss,val_acc,loss,acc
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning
/home/neelr/anaconda3/lib/python3.6/site-packages/keras/callbacks.py:958: RuntimeWarning: Reduce LR on plateau conditioned on metric `accuracy` which is not available. Available metrics are: val_loss,val_acc,loss,acc,lr
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


50/50 [==============================] - 4s 74ms/step - loss: 0.5932 - acc: 0.7774 - val_loss: 1.9309 - val_acc: 0.6199
Epoch 3/10
 3/50 [>.............................] - ETA: 2s - loss: 0.3979 - acc: 1.0000

/home/neelr/anaconda3/lib/python3.6/site-packages/keras/callbacks.py:526: RuntimeWarning: Early stopping conditioned on metric `accuracy` which is not available. Available metrics are: val_loss,val_acc,loss,acc,lr
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


50/50 [==============================] - 4s 75ms/step - loss: 0.3996 - acc: 0.9229 - val_loss: 2.3314 - val_acc: 0.5709
Epoch 4/10
50/50 [==============================] - 4s 76ms/step - loss: 0.0963 - acc: 0.9665 - val_loss: 2.3451 - val_acc: 0.5582
Epoch 5/10
50/50 [==============================] - 4s 88ms/step - loss: 0.1007 - acc: 0.9795 - val_loss: 1.8024 - val_acc: 0.6988
Epoch 6/10
50/50 [==============================] - 4s 86ms/step - loss: 0.4062 - acc: 0.9154 - val_loss: 2.8135 - val_acc: 0.5257
Epoch 7/10
50/50 [==============================] - 4s 88ms/step - loss: 0.1236 - acc: 0.9681 - val_loss: 2.2378 - val_acc: 0.6370
Epoch 8/10
50/50 [==============================] - 5s 94ms/step - loss: 0.0445 - acc: 0.9920 - val_loss: 2.3630 - val_acc: 0.5940
Epoch 9/10
50/50 [==============================] - 5s 97ms/step - loss: 0.2338 - acc: 0.9493 - val_loss: 1.1811 - val_acc: 0.7768
Epoch 10/10
50/50 [==============================] - 5s 91ms/step - loss: 0.0607 - acc: 0.9785

In [20]:
from keras.layers import Flatten, Dropout
model1 = Sequential()
model1.add(LSTM(2048, return_sequences=False, input_shape= (15, 1000), dropout=0.5))
model1.add(Dense(512, activation='relu'))
model1.add(Dropout(0.5))
#model1.add(Flatten())
model1.add(Dense(len(classes), activation='softmax'))

In [21]:
model1.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics=['accuracy'])

In [22]:
print(model1.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, 2048)              24977408  
_________________________________________________________________
dense_2 (Dense)              (None, 512)               1049088   
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 9)                 4617      
Total params: 26,031,113
Trainable params: 26,031,113
Non-trainable params: 0
_________________________________________________________________
None


In [23]:
epochs = 10
model1.fit_generator(generator=train_generator, epochs = 10, steps_per_epoch=50, callbacks=callbacks, verbose=1,
                   validation_data=test_generator, validation_steps=50)

Epoch 1/10
50/50 [==============================] - 135s 3s/step - loss: 2.5396 - acc: 0.5073 - val_loss: 4.3395 - val_acc: 0.1874
Epoch 2/10


/home/neelr/anaconda3/lib/python3.6/site-packages/keras/callbacks.py:526: RuntimeWarning: Early stopping conditioned on metric `accuracy` which is not available. Available metrics are: val_loss,val_acc,loss,acc
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning
/home/neelr/anaconda3/lib/python3.6/site-packages/keras/callbacks.py:958: RuntimeWarning: Reduce LR on plateau conditioned on metric `accuracy` which is not available. Available metrics are: val_loss,val_acc,loss,acc,lr
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


50/50 [==============================] - 145s 3s/step - loss: 5.3349 - acc: 0.1789 - val_loss: 7.7561 - val_acc: 0.0865
Epoch 3/10


/home/neelr/anaconda3/lib/python3.6/site-packages/keras/callbacks.py:526: RuntimeWarning: Early stopping conditioned on metric `accuracy` which is not available. Available metrics are: val_loss,val_acc,loss,acc,lr
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


50/50 [==============================] - 145s 3s/step - loss: 7.2416 - acc: 0.2215 - val_loss: 9.0505 - val_acc: 0.1907
Epoch 4/10
50/50 [==============================] - 149s 3s/step - loss: 8.4694 - acc: 0.3184 - val_loss: 11.5332 - val_acc: 0.2481
Epoch 5/10
50/50 [==============================] - 144s 3s/step - loss: 12.4203 - acc: 0.2021 - val_loss: 12.6373 - val_acc: 0.1779
Epoch 6/10
50/50 [==============================] - 148s 3s/step - loss: 12.0224 - acc: 0.1929 - val_loss: 11.1257 - val_acc: 0.3074
Epoch 7/10
50/50 [==============================] - 142s 3s/step - loss: 11.7670 - acc: 0.2320 - val_loss: 10.8447 - val_acc: 0.3276
Epoch 8/10
50/50 [==============================] - 124s 2s/step - loss: 12.3159 - acc: 0.1277 - val_loss: 9.6755 - val_acc: 0.3921
Epoch 9/10
50/50 [==============================] - 134s 3s/step - loss: 11.3103 - acc: 0.2696 - val_loss: 11.9523 - val_acc: 0.2306
Epoch 10/10
50/50 [==============================] - 129s 3s/step - loss: 8.4003 - a

In [24]:
from keras.layers import Flatten, Dropout
model2 = Sequential()
model2.add(LSTM(2048, return_sequences=True, input_shape= (15, 1000), dropout=0.5))
model2.add(Dense(512, activation='relu'))
model2.add(Dropout(0.5))
model2.add(Flatten())
model2.add(Dense(len(classes), activation='softmax'))
model2.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics=['accuracy'])
print(model2.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_4 (LSTM)                (None, 15, 2048)          24977408  
_________________________________________________________________
dense_4 (Dense)              (None, 15, 512)           1049088   
_________________________________________________________________
dropout_2 (Dropout)          (None, 15, 512)           0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 7680)              0         
_________________________________________________________________
dense_5 (Dense)              (None, 9)                 69129     
Total params: 26,095,625
Trainable params: 26,095,625
Non-trainable params: 0
_________________________________________________________________
None


In [25]:
epochs = 10
model2.fit_generator(generator=train_generator, epochs = 10, steps_per_epoch=100, callbacks=callbacks, verbose=1,
                   validation_data=test_generator, validation_steps=50)

Epoch 1/10
100/100 [==============================] - 333s 3s/step - loss: 1.3855 - acc: 0.5412 - val_loss: 5.4876 - val_acc: 0.2492
Epoch 2/10


/home/neelr/anaconda3/lib/python3.6/site-packages/keras/callbacks.py:526: RuntimeWarning: Early stopping conditioned on metric `accuracy` which is not available. Available metrics are: val_loss,val_acc,loss,acc
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning
/home/neelr/anaconda3/lib/python3.6/site-packages/keras/callbacks.py:958: RuntimeWarning: Reduce LR on plateau conditioned on metric `accuracy` which is not available. Available metrics are: val_loss,val_acc,loss,acc,lr
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


100/100 [==============================] - 442s 4s/step - loss: 1.1922 - acc: 0.7384 - val_loss: 1.1249 - val_acc: 0.7322
Epoch 3/10


/home/neelr/anaconda3/lib/python3.6/site-packages/keras/callbacks.py:526: RuntimeWarning: Early stopping conditioned on metric `accuracy` which is not available. Available metrics are: val_loss,val_acc,loss,acc,lr
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


100/100 [==============================] - 268s 3s/step - loss: 0.7324 - acc: 0.7806 - val_loss: 1.5842 - val_acc: 0.6275
Epoch 4/10
100/100 [==============================] - 248s 2s/step - loss: 0.4101 - acc: 0.8669 - val_loss: 4.0786 - val_acc: 0.4617
Epoch 5/10
100/100 [==============================] - 254s 3s/step - loss: 0.4833 - acc: 0.8736 - val_loss: 2.3168 - val_acc: 0.5021
Epoch 6/10
100/100 [==============================] - 262s 3s/step - loss: 0.2411 - acc: 0.9196 - val_loss: 2.0919 - val_acc: 0.5192
Epoch 7/10
100/100 [==============================] - 264s 3s/step - loss: 0.3104 - acc: 0.9108 - val_loss: 2.9323 - val_acc: 0.5697
Epoch 8/10
100/100 [==============================] - 240s 2s/step - loss: 0.2828 - acc: 0.9173 - val_loss: 2.4788 - val_acc: 0.6148
Epoch 9/10
100/100 [==============================] - 232s 2s/step - loss: 0.2784 - acc: 0.9186 - val_loss: 3.2861 - val_acc: 0.5428
Epoch 10/10
100/100 [==============================] - 231s 2s/step - loss: 0.22

In [26]:
from keras.layers import Flatten, Dropout
model3 = Sequential()
model3.add(LSTM(2048, return_sequences=True, input_shape= (15, 1000), dropout=0.5))
model3.add(LSTM(2048, return_sequences=True))
model3.add(Dense(512, activation='relu'))
model3.add(Dropout(0.5))
model3.add(Flatten())
model3.add(Dense(len(classes), activation='softmax'))
model3.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics=['accuracy'])
print(model3.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_5 (LSTM)                (None, 15, 2048)          24977408  
_________________________________________________________________
lstm_6 (LSTM)                (None, 15, 2048)          33562624  
_________________________________________________________________
dense_6 (Dense)              (None, 15, 512)           1049088   
_________________________________________________________________
dropout_3 (Dropout)          (None, 15, 512)           0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 7680)              0         
_________________________________________________________________
dense_7 (Dense)              (None, 9)                 69129     
Total params: 59,658,249
Trainable params: 59,658,249
Non-trainable params: 0
________________________________________________________________

In [27]:
epochs = 10
model3.fit_generator(generator=train_generator, epochs = 10, steps_per_epoch=100, callbacks=callbacks, verbose=1,
                   validation_data=test_generator, validation_steps=50)

Epoch 1/10
100/100 [==============================] - 547s 5s/step - loss: 1.6525 - acc: 0.5590 - val_loss: 1.5374 - val_acc: 0.4247
Epoch 2/10


/home/neelr/anaconda3/lib/python3.6/site-packages/keras/callbacks.py:526: RuntimeWarning: Early stopping conditioned on metric `accuracy` which is not available. Available metrics are: val_loss,val_acc,loss,acc
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning
/home/neelr/anaconda3/lib/python3.6/site-packages/keras/callbacks.py:958: RuntimeWarning: Reduce LR on plateau conditioned on metric `accuracy` which is not available. Available metrics are: val_loss,val_acc,loss,acc,lr
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


100/100 [==============================] - 543s 5s/step - loss: 1.1376 - acc: 0.6180 - val_loss: 1.7569 - val_acc: 0.4099
Epoch 3/10


/home/neelr/anaconda3/lib/python3.6/site-packages/keras/callbacks.py:526: RuntimeWarning: Early stopping conditioned on metric `accuracy` which is not available. Available metrics are: val_loss,val_acc,loss,acc,lr
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


100/100 [==============================] - 543s 5s/step - loss: 0.6685 - acc: 0.8006 - val_loss: 2.5745 - val_acc: 0.5122
Epoch 4/10
100/100 [==============================] - 549s 5s/step - loss: 0.7193 - acc: 0.7918 - val_loss: 1.9346 - val_acc: 0.4742
Epoch 5/10
100/100 [==============================] - 548s 5s/step - loss: 0.5103 - acc: 0.8503 - val_loss: 1.9982 - val_acc: 0.5487
Epoch 6/10
100/100 [==============================] - 560s 6s/step - loss: 0.5369 - acc: 0.8276 - val_loss: 2.9626 - val_acc: 0.6286
Epoch 7/10
100/100 [==============================] - 562s 6s/step - loss: 0.5289 - acc: 0.8419 - val_loss: 4.6216 - val_acc: 0.5222
Epoch 8/10
100/100 [==============================] - 553s 6s/step - loss: 0.2436 - acc: 0.9211 - val_loss: 3.6618 - val_acc: 0.5388
Epoch 9/10
100/100 [==============================] - 553s 6s/step - loss: 0.1776 - acc: 0.9518 - val_loss: 3.9813 - val_acc: 0.5658
Epoch 10/10
100/100 [==============================] - 549s 5s/step - loss: 0.23

In [30]:
model.save('model3.h5')

In [28]:
from keras.layers import Flatten, Dropout
model4 = Sequential()
model4.add(LSTM(2048, return_sequences=True, input_shape= (15, 1000), dropout=0.5))
model4.add(LSTM(2048, return_sequences=True))
model4.add(Dense(512, activation='relu'))
model4.add(Dropout(0.5))
model4.add(Flatten())
model4.add(Dense(len(classes), activation='softmax'))
model4.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics=['accuracy'])
print(model4.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_7 (LSTM)                (None, 15, 2048)          24977408  
_________________________________________________________________
lstm_8 (LSTM)                (None, 15, 2048)          33562624  
_________________________________________________________________
dense_8 (Dense)              (None, 15, 512)           1049088   
_________________________________________________________________
dropout_4 (Dropout)          (None, 15, 512)           0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 7680)              0         
_________________________________________________________________
dense_9 (Dense)              (None, 9)                 69129     
Total params: 59,658,249
Trainable params: 59,658,249
Non-trainable params: 0
________________________________________________________________

In [29]:
model4.fit_generator(generator=train_generator, epochs = 20, steps_per_epoch=100, callbacks=callbacks, verbose=1,
                   validation_data=test_generator, validation_steps=50)

Epoch 1/20
 17/100 [====>.........................] - ETA: 6:21 - loss: 1.9119 - acc: 0.4065

KeyboardInterrupt: 

In [19]:
from keras.layers import Flatten, Dropout
model5 = Sequential()
model5.add(LSTM(128, return_sequences=True, input_shape= (frames_considered, 1000), dropout=0.5))
model5.add(LSTM(128, return_sequences=True))
model5.add(Dense(100, activation='relu'))
model5.add(Dropout(0.5))
model5.add(Flatten())
model5.add(Dense(len(classes), activation='softmax'))
model5.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics=['accuracy'])
print(model5.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, 20, 128)           578048    
_________________________________________________________________
lstm_4 (LSTM)                (None, 20, 128)           131584    
_________________________________________________________________
dense_3 (Dense)              (None, 20, 100)           12900     
_________________________________________________________________
dropout_2 (Dropout)          (None, 20, 100)           0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 2000)              0         
_________________________________________________________________
dense_4 (Dense)              (None, 9)                 18009     
Total params: 740,541
Trainable params: 740,541
Non-trainable params: 0
_________________________________________________________________
None

In [20]:
model5.fit_generator(generator=train_generator, epochs = 20, steps_per_epoch=100, callbacks=callbacks, verbose=1,
                   validation_data=test_generator, validation_steps=50)

Epoch 1/20
100/100 [==============================] - 11s 113ms/step - loss: 1.6563 - acc: 0.4707 - val_loss: 2.1385 - val_acc: 0.4664
Epoch 2/20
  1/100 [..............................] - ETA: 5s - loss: 0.7004 - acc: 0.8000

/home/neelr/anaconda3/lib/python3.6/site-packages/keras/callbacks.py:526: RuntimeWarning: Early stopping conditioned on metric `accuracy` which is not available. Available metrics are: val_loss,val_acc,loss,acc
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning
/home/neelr/anaconda3/lib/python3.6/site-packages/keras/callbacks.py:958: RuntimeWarning: Reduce LR on plateau conditioned on metric `accuracy` which is not available. Available metrics are: val_loss,val_acc,loss,acc,lr
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


100/100 [==============================] - 9s 92ms/step - loss: 1.0958 - acc: 0.6512 - val_loss: 2.1781 - val_acc: 0.5046
Epoch 3/20
  2/100 [..............................] - ETA: 7s - loss: 0.3944 - acc: 0.8944

/home/neelr/anaconda3/lib/python3.6/site-packages/keras/callbacks.py:526: RuntimeWarning: Early stopping conditioned on metric `accuracy` which is not available. Available metrics are: val_loss,val_acc,loss,acc,lr
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


100/100 [==============================] - 9s 93ms/step - loss: 0.9230 - acc: 0.6938 - val_loss: 1.6407 - val_acc: 0.6449
Epoch 4/20
100/100 [==============================] - 10s 99ms/step - loss: 0.5394 - acc: 0.8371 - val_loss: 1.6138 - val_acc: 0.6874
Epoch 5/20
100/100 [==============================] - 10s 100ms/step - loss: 0.3489 - acc: 0.8881 - val_loss: 1.8025 - val_acc: 0.7205
Epoch 6/20
100/100 [==============================] - 11s 107ms/step - loss: 0.3221 - acc: 0.9056 - val_loss: 1.3738 - val_acc: 0.7902
Epoch 7/20
100/100 [==============================] - 11s 109ms/step - loss: 0.2267 - acc: 0.9267 - val_loss: 1.8170 - val_acc: 0.7414
Epoch 8/20
100/100 [==============================] - 11s 109ms/step - loss: 0.3751 - acc: 0.8893 - val_loss: 1.6296 - val_acc: 0.7601
Epoch 9/20
100/100 [==============================] - 11s 106ms/step - loss: 0.2378 - acc: 0.9257 - val_loss: 2.1758 - val_acc: 0.7196
Epoch 10/20
100/100 [==============================] - 11s 106ms/step

In [21]:
#model.save('model.h5')
#model1.save('model1.h5')
#model2.save('model2.h5')
#model3.save('model3.h5')
#model4.save('model4.h5')
model5.save('model5new.h5')

In [24]:
from keras.layers import Flatten, Dropout
model6 = Sequential()
model6.add(LSTM(100, return_sequences=True, input_shape= (frames_considered, 1000), dropout=0.5))
model6.add(LSTM(100, return_sequences=True))
model6.add(Dense(100, activation='relu'))
model6.add(Dropout(0.5))
model6.add(Flatten())
model6.add(Dense(len(classes), activation='softmax'))
model6.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics=['accuracy'])
print(model6.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_7 (LSTM)                (None, 20, 100)           440400    
_________________________________________________________________
lstm_8 (LSTM)                (None, 20, 100)           80400     
_________________________________________________________________
dense_7 (Dense)              (None, 20, 100)           10100     
_________________________________________________________________
dropout_4 (Dropout)          (None, 20, 100)           0         
_________________________________________________________________
flatten_4 (Flatten)          (None, 2000)              0         
_________________________________________________________________
dense_8 (Dense)              (None, 9)                 18009     
Total params: 548,909
Trainable params: 548,909
Non-trainable params: 0
_________________________________________________________________
None

In [25]:
model6.fit_generator(generator=train_generator, epochs = 20, steps_per_epoch=100, callbacks=callbacks, verbose=1,
                   validation_data=test_generator, validation_steps=50)

Epoch 1/20
100/100 [==============================] - 10s 98ms/step - loss: 1.4899 - acc: 0.4834 - val_loss: 1.6642 - val_acc: 0.5504
Epoch 2/20
  1/100 [..............................] - ETA: 4s - loss: 1.3052 - acc: 0.5556

/home/neelr/anaconda3/lib/python3.6/site-packages/keras/callbacks.py:526: RuntimeWarning: Early stopping conditioned on metric `accuracy` which is not available. Available metrics are: val_loss,val_acc,loss,acc
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning
/home/neelr/anaconda3/lib/python3.6/site-packages/keras/callbacks.py:958: RuntimeWarning: Reduce LR on plateau conditioned on metric `accuracy` which is not available. Available metrics are: val_loss,val_acc,loss,acc,lr
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


100/100 [==============================] - 8s 84ms/step - loss: 1.0380 - acc: 0.6845 - val_loss: 1.3860 - val_acc: 0.6276
Epoch 3/20
  2/100 [..............................] - ETA: 7s - loss: 0.2106 - acc: 0.9615

/home/neelr/anaconda3/lib/python3.6/site-packages/keras/callbacks.py:526: RuntimeWarning: Early stopping conditioned on metric `accuracy` which is not available. Available metrics are: val_loss,val_acc,loss,acc,lr
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


100/100 [==============================] - 9s 88ms/step - loss: 0.8739 - acc: 0.7262 - val_loss: 1.4569 - val_acc: 0.7491
Epoch 4/20
100/100 [==============================] - 10s 96ms/step - loss: 0.4927 - acc: 0.8527 - val_loss: 1.1972 - val_acc: 0.7371
Epoch 5/20
100/100 [==============================] - 10s 101ms/step - loss: 0.5129 - acc: 0.8232 - val_loss: 1.0416 - val_acc: 0.6424
Epoch 6/20
100/100 [==============================] - 10s 98ms/step - loss: 0.3570 - acc: 0.8766 - val_loss: 0.9391 - val_acc: 0.8492
Epoch 7/20
100/100 [==============================] - 10s 102ms/step - loss: 0.2458 - acc: 0.9248 - val_loss: 1.1210 - val_acc: 0.7042
Epoch 8/20
100/100 [==============================] - 10s 97ms/step - loss: 0.0961 - acc: 0.9678 - val_loss: 1.8169 - val_acc: 0.6220
Epoch 9/20
100/100 [==============================] - 10s 99ms/step - loss: 0.1248 - acc: 0.9584 - val_loss: 2.5191 - val_acc: 0.5624
Epoch 10/20
100/100 [==============================] - 11s 107ms/step - 

In [ ]:
model6.save('model6.h5')

In [17]:
from keras.layers import Flatten, Dropout
model7 = Sequential()
model7.add(LSTM(128, return_sequences=True, input_shape= (frames_considered, 1000), dropout=0.5))
model7.add(LSTM(128, return_sequences=True))
model7.add(Dense(100, activation='relu'))
model7.add(Dropout(0.5))
model7.add(Flatten())
model7.add(Dense(len(classes), activation='softmax'))
model7.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics=['accuracy'])
print(model7.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 20, 128)           578048    
_________________________________________________________________
lstm_2 (LSTM)                (None, 20, 128)           131584    
_________________________________________________________________
dense_1 (Dense)              (None, 20, 100)           12900     
_________________________________________________________________
dropout_1 (Dropout)          (None, 20, 100)           0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 2000)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 9)                 18009     
Total params: 740,541
Trainable params: 740,541
Non-trainable params: 0
_________________________________________________________________
None

In [19]:
model7.fit_generator(generator=train_generator, epochs = 20, steps_per_epoch=100, callbacks=callbacks, verbose=1,
                   validation_data=test_generator, validation_steps=50)

Epoch 1/20
100/100 [==============================] - 10s 102ms/step - loss: 1.6315 - acc: 0.4165 - val_loss: 0.9998 - val_acc: 0.5737
Epoch 2/20
  1/100 [..............................] - ETA: 5s - loss: 1.6672 - acc: 0.4000

/home/neelr/anaconda3/lib/python3.6/site-packages/keras/callbacks.py:526: RuntimeWarning: Early stopping conditioned on metric `accuracy` which is not available. Available metrics are: val_loss,val_acc,loss,acc
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning
/home/neelr/anaconda3/lib/python3.6/site-packages/keras/callbacks.py:958: RuntimeWarning: Reduce LR on plateau conditioned on metric `accuracy` which is not available. Available metrics are: val_loss,val_acc,loss,acc,lr
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


100/100 [==============================] - 9s 94ms/step - loss: 1.2511 - acc: 0.5782 - val_loss: 1.4799 - val_acc: 0.6517
Epoch 3/20
  2/100 [..............................] - ETA: 7s - loss: 1.7453 - acc: 0.3111

/home/neelr/anaconda3/lib/python3.6/site-packages/keras/callbacks.py:526: RuntimeWarning: Early stopping conditioned on metric `accuracy` which is not available. Available metrics are: val_loss,val_acc,loss,acc,lr
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


100/100 [==============================] - 10s 105ms/step - loss: 0.8083 - acc: 0.7027 - val_loss: 1.5332 - val_acc: 0.5721
Epoch 4/20
100/100 [==============================] - 12s 116ms/step - loss: 0.4680 - acc: 0.8477 - val_loss: 1.3911 - val_acc: 0.8074
Epoch 5/20
100/100 [==============================] - 11s 113ms/step - loss: 0.4322 - acc: 0.8487 - val_loss: 0.2547 - val_acc: 0.9610
Epoch 6/20
100/100 [==============================] - 11s 109ms/step - loss: 0.4339 - acc: 0.8478 - val_loss: 0.8750 - val_acc: 0.8459
Epoch 7/20
100/100 [==============================] - 11s 115ms/step - loss: 0.3654 - acc: 0.8690 - val_loss: 0.8294 - val_acc: 0.8921
Epoch 8/20
100/100 [==============================] - 11s 106ms/step - loss: 0.3971 - acc: 0.8767 - val_loss: 0.5764 - val_acc: 0.7173
Epoch 9/20
100/100 [==============================] - 9s 94ms/step - loss: 0.6500 - acc: 0.8115 - val_loss: 0.8756 - val_acc: 0.8369
Epoch 10/20
100/100 [==============================] - 10s 100ms/ste

In [20]:
model7.save('90-10.h5')